In [1]:
import joblib
import os
from numpy.linalg import inv
from API.utils import *
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

In [2]:
os.listdir('./model')

['ANN(58_to_sp162).pkl',
 'ANN(58_to_y162).pkl',
 'ANN(energy).pkl',
 'ANN(tray).pkl',
 'eva.pkl',
 'OperationOptimModel.pkl',
 'SVR(4_to_23).pkl',
 'transformer(23_to_54).pkl',
 'transformer(33_to_15).pkl',
 'transformer(43_to_33).pkl',
 'transformer(54_to_23).pkl',
 'transformer(54_to_33).pkl',
 'transformer(SP162_to_Y162).pkl']

In [3]:
os.listdir('./data/phase_2/cleaned')

['col_names.pkl', 'train(energy).csv', 'train.csv']

# load pretrained model

In [4]:
A = joblib.load('./model/SVR(4_to_23).pkl')
B = joblib.load('./model/transformer(23_to_54).pkl')
C = joblib.load('./model/ANN(58_to_sp162).pkl')
D = joblib.load('./model/transformer(SP162_to_Y162).pkl')
E = joblib.load('./model/ANN(energy).pkl')
F = joblib.load('./model/transformer(54_to_33).pkl')
G = joblib.load('./model/transformer(33_to_15).pkl')
H = joblib.load('./model/transformer(43_to_33).pkl')
I = joblib.load('./model/transformer(54_to_23).pkl')
col_names = joblib.load('./data/phase_2/cleaned/col_names.pkl')

In [5]:
col_names.keys()

dict_keys(['xna', 'xle', 'xhc', 'xhe', 'fna', 'fle', 'fhc', 'fhe', 'yld', 'Tray_temp', 'duty', 'Rate_m3', 'Rate_ton', 'sle', 'shc', 'she', 'case'])

# A

In [6]:
A_X = pd.read_csv('./data/phase_1/test_170.csv',index_col=0).sample(1)[['T10','T50','T90','N+A']]
A_X.head()

T10    T50    T90     N+A
139  99.9  119.9  146.9  41.307

In [7]:
A_pred = A.predict(A_X)
print(A_pred.sum(axis=1))
A_pred

0    100.0
dtype: float64


C5NP      C5IP       C5N      C6NP      C6IP       C6N       C6A  \
0  1.054952  0.713361  0.153138  3.288574  2.058669  3.550635  0.645131   

      C7NP      C7IP       C7N       C7A      C8NP      C8IP       C8N  \
0  7.78893  6.232049  9.894674  3.337994  7.951538  8.336734  7.354495   

        C8A     C9NP      C9IP       C9N       C9A     C10NP     C10IP  \
0  6.043764  5.17344  9.482511  5.116143  3.679991  1.219736  5.392507   

       C10N      C10A  
0  0.730878  0.800157

# B

In [8]:
B_pred = B(A_pred)
print(B_pred.sum(axis=1))
B_pred.head()

0    100.0
dtype: float64


O2    METHANE    ETHANE    PROPANE    BUTANE    IPENTANE    PENTANE  \
0   0.0        0.0       0.0   0.035668  0.071336     0.59209   1.054952   

     T2PENTEN        CP       3MP    T2HEXENE    HEXANE       MCP      22MP  \
0    0.014267  0.153138  2.017495    0.041173  3.288574  2.303498  0.596948   

     BENZENE        CH     223MB    4M1HEXEN      2MHX      1T3M    HEPTANE  \
0   0.645131  1.247137  0.655922    0.013084  4.966095  0.915905    7.78893   

     H2O       MCH      ECP    23DM1HX    TOLUENE    2MHEPT    OCTANE  \
0    0.0  5.305518  3.67325   0.083367   3.337994  8.253367  7.951538   

        ECH    EBENZENE    PXYLENE    MXYLENE    OXYLENE    2MOCTN    1NONENE  \
0  7.354495    1.435122   0.814092   2.009385   1.785166   9.24163   0.240881   

     NONANE    CUMENE    1T35TMCH       PCH    PRBENZEN    135MBENZ  \
0   5.17344  0.052158    3.069545  2.046597    0.222474    0.955408   

     1M2EBNZN    123MBENZ    24DMOCT      TEMH       3MN    TBCYHXAN  \
0    1.411984    1.037967   0.906823  1.345499  3.140185    0.730878   

     DECANE     ODEBZ    UNDECANE    PNBZ    DODECANE    HXBNZN    TRIDECAN  
0  1.022837  0.748226    0.193451  0.0435    0.003449   0.00843         0.0

# B2 54 to 23

In [9]:
B.W.shape

(23, 54)

# C

In [10]:
case = pd.read_csv('./data/phase_2/cleaned/train.csv',index_col=0)[col_names['case']].sample(1).reset_index(drop=True)
C_X = case.join(B_pred)
C_X

Case Conditions_Feed Rate to C013 (Input)_m3/hr  \
0                                             20.0   

   Case Conditions_C7+ in Light End (Input)_wt%  \
0                                          10.0   

   Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr  \
0                                              100.0    

   Case Conditions_C6P- in Heart Cut (Input)_wt%    O2    METHANE    ETHANE  \
0                                            1.5   0.0        0.0       0.0   

     PROPANE    BUTANE    IPENTANE    PENTANE    T2PENTEN        CP       3MP  \
0   0.035668  0.071336     0.59209   1.054952    0.014267  0.153138  2.017495   

     T2HEXENE    HEXANE       MCP      22MP    BENZENE        CH     223MB  \
0    0.041173  3.288574  2.303498  0.596948   0.645131  1.247137  0.655922   

     4M1HEXEN      2MHX      1T3M    HEPTANE    H2O       MCH      ECP  \
0    0.013084  4.966095  0.915905    7.78893    0.0  5.305518  3.67325   

     23DM1HX    TOLUENE    2MHEPT    OCTANE       ECH    EBENZENE    PXYLENE  \
0   0.083367   3.337994  8.253367  7.951538  7.354495    1.435122   0.814092   

     MXYLENE    OXYLENE    2MOCTN    1NONENE    NONANE    CUMENE    1T35TMCH  \
0   2.009385   1.785166   9.24163   0.240881   5.17344  0.052158    3.069545   

        PCH    PRBENZEN    135MBENZ    1M2EBNZN    123MBENZ    24DMOCT  \
0  2.046597    0.222474    0.955408    1.411984    1.037967   0.906823   

       TEMH       3MN    TBCYHXAN    DECANE     ODEBZ    UNDECANE    PNBZ  \
0  1.345499  3.140185    0.730878  1.022837  0.748226    0.193451  0.0435   

     DODECANE    HXBNZN    TRIDECAN  
0    0.003449   0.00843         0.0

In [11]:
C_pred = C.predict(C_X)
print(C_pred.sum(axis=1))
C_pred

[[9.99764860e-01 3.16569989e-04 2.89963122e-04 9.99631882e-01
  1.98322203e-04 2.36381791e-04 9.99775112e-01 2.26373508e-04
  2.54330196e-04 9.89390314e-01 1.05778323e-02 1.10683664e-04
  9.77368951e-01 2.23548524e-02 7.06154970e-05 9.56218898e-01
  4.21928540e-02 5.95514684e-05 9.44880903e-01 5.44991009e-02
  1.35346709e-04 9.49462891e-01 5.30663654e-02 1.03676000e-04
  9.08061922e-01 9.12098289e-02 1.01135724e-04 8.44387889e-01
  1.53744787e-01 7.92755091e-05 7.06775248e-01 2.97280520e-01
  3.93984155e-05 7.48252213e-01 2.48875275e-01 6.96869174e-05
  6.14068925e-01 3.80927294e-01 2.97770330e-05 4.34244663e-01
  5.61812580e-01 8.65378461e-05 5.72579980e-01 4.28726554e-01
  2.35613377e-04 4.17436838e-01 5.81060350e-01 1.20004239e-04
  3.43997002e-01 6.51580513e-01 1.60790936e-04 1.74383685e-01
  8.33249450e-01 1.27568055e-04 7.10266978e-02 9.29326355e-01
  6.59439611e-05 3.69998030e-02 9.63623345e-01 2.11677572e-04
  3.89123918e-03 9.98588264e-01 1.15099429e-04 9.99701023e-01
  2.0852

Individual Component to Light End Split Factor_Oxygen_Ratio  \
0                                           0.999394             

   Individual Component to Heart Cut Split Factor_Oxygen_Ratio  \
0                                           0.000316             

   Individual Component to Heavy End Split Factor_Oxygen_Ratio  \
0                                            0.00029             

   Individual Component to Light End Split Factor_Methane_Ratio  \
0                                           0.999565              

   Individual Component to Heart Cut Split Factor_Methane_Ratio  \
0                                           0.000198              

   Individual Component to Heavy End Split Factor_Methane_Ratio  \
0                                           0.000236              

   Individual Component to Light End Split Factor_Ethane_Ratio  \
0                                            0.99952             

   Individual Component to Heart Cut Split Factor_Ethane_Ratio  \
0                                           0.000226             

   Individual Component to Heavy End Split Factor_Ethane_Ratio  \
0                                           0.000254             

   Individual Component to Light End Split Factor_Propane_Ratio  \
0                                           0.989312              

   Individual Component to Heart Cut Split Factor_Propane_Ratio  \
0                                           0.010577              

   Individual Component to Heavy End Split Factor_Propane_Ratio  \
0                                           0.000111              

   Individual Component to Light End Split Factor_n-Butane_Ratio  \
0                                            0.97757               

   Individual Component to Heart Cut Split Factor_n-Butane_Ratio  \
0                                           0.022359               

   Individual Component to Heavy End Split Factor_n-Butane_Ratio  \
0                                           0.000071               

   Individual Component to Light End Split Factor_i-Pentane_Ratio  \
0                                           0.957683                

   Individual Component to Heart Cut Split Factor_i-Pentane_Ratio  \
0                                           0.042257                

   Individual Component to Heavy End Split Factor_i-Pentane_Ratio  \
0                                            0.00006                

   Individual Component to Light End Split Factor_n-Pentane_Ratio  \
0                                           0.945339                

   Individual Component to Heart Cut Split Factor_n-Pentane_Ratio  \
0                                           0.054526                

   Individual Component to Heavy End Split Factor_n-Pentane_Ratio  \
0                                           0.000135                

   Individual Component to Light End Split Factor_tr2-Pentene_Ratio  \
0                                            0.94697                  

   Individual Component to Heart Cut Split Factor_tr2-Pentene_Ratio  \
0                                           0.052927                  

   Individual Component to Heavy End Split Factor_tr2-Pentene_Ratio  \
0                                           0.000103                  

   Individual Component to Light End Split Factor_Cyclopentane_Ratio  \
0                                           0.908632                   

   Individual Component to Heart Cut Split Factor_Cyclopentane_Ratio  \
0                                           0.091267                   

   Individual Component to Heavy End Split Factor_Cyclopentane_Ratio  \
0                                           0.000101                   

   Individual Component to Light End Split Factor_3-Methylpentane_Ratio  \
0                                             0.8459                      

   Individual Component to Heart Cut Split Factor_3-Methylpentane_Ratio  \
0                                            0.15402        

# D

In [12]:
D_pred = D(B_pred,C_pred)
print(D_pred.sum(axis=1))
D_pred

0    300.0
dtype: float64


Light End Product Properties_Oxygen_wt%  \
0                                      0.0   

   Light End Product Properties_Methane_wt%  \
0                                       0.0   

   Light End Product Properties_Ethane_wt%  \
0                                      0.0   

   Light End Product Properties_Propane_wt%  \
0                                  0.381033   

   Light End Product Properties_n-Butane_wt%  \
0                                   0.753021   

   Light End Product Properties_i-Pentane_wt%  \
0                                    6.122924   

   Light End Product Properties_n-Pentane_wt%  \
0                                   10.768863   

   Light End Product Properties_tr2-Pentene_wt%  \
0                                       0.14589   

   Light End Product Properties_Cyclopentane_wt%  \
0                                       1.502521   

   Light End Product Properties_3-Methylpentane_wt%  \
0                                          18.42814   

   Light End Product Properties_tr2-Hexene_wt%  \
0                                     0.312948   

   Light End Product Properties_n-Hexane_wt%  \
0                                  26.645526   

   Light End Product Properties_Methylcyclopentane_wt%  \
0                                           15.35042     

   Light End Product Properties_2,2-Dimethylpentane_wt%  \
0                                            2.80995      

   Light End Product Properties_Benzene_wt%  \
0                                  3.982581   

   Light End Product Properties_Cyclohexane_wt%  \
0                                      5.629316   

   Light End Product Properties_2,2,3-Trimethylbutane_wt%  \
0                                           2.446873        

   Light End Product Properties_4-Methyl-1-hexene_wt%  \
0                                           0.024448    

   Light End Product Properties_2-Methylhexane_wt%  \
0                                         3.807189   

   Light End Product Properties_t-1,3-Dimethylcyclopentane_wt%  \
0                                           0.365626             

   Light End Product Properties_n-Heptane_wt%  \
0                                     0.32643   

   Light End Product Properties_H2O_wt%  \
0                                   0.0   

   Light End Product Properties_Methylcyclohexane_wt%  \
0                                           0.024175    

   Light End Product Properties_Ethylcyclopentane_wt%  \
0                                            0.02404    

   Light End Product Properties_2,3-Dimethyl-1-hexene_wt%  \
0                                           0.000275        

   Light End Product Properties_Toluene_wt%  \
0                                  0.010882   

   Light End Product Properties_2-Methylheptane_wt%  \
0                                          0.016383   

   Light End Product Properties_n-Octane_wt%  \
0                                   0.032836   

   Light End Product Properties_Ethylcyclohexane_wt%  \
0                                           0.012393   

   Light End Product Properties_EthylBenzene_wt%  \
0                                       0.006441   

   Light End Product Properties_P-Xylene_wt%  \
0                                   0.002406   

   Light End Product Properties_M-Xylene_wt%  \
0                                   0.004711   

   Light End Product Properties_O-Xylene_wt%  \
0                                    0.00583   

   Light End Product Properties_2-Methyloctane_wt%  \
0                                         0.032037   

   Light End Product Properties_1-Nonene_wt%  \
0                                   0.000227   

   Light End Product Properties_n-Nonane_wt%  \
0                                   0.005952   

   Light End Product Properties_i-Propylbenzene_wt%  \
0                                          0.000104   

   Light End Product Properties_1-trans-3,5-Trimethylcyclohexane_wt%  \
0                                           0.003525            

# E

In [13]:
df = pd.read_csv('./data/phase_2/cleaned/train(energy).csv',index_col=0).dropna(axis=0).sample(1)
E_X = df[E.x_col]
E_X

Case Conditions_Feed Rate to C013 (Input)_m3/hr  \
097-02                                             10.0   

        Case Conditions_C7+ in Light End (Input)_wt%  \
097-02                                           5.0   

        Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr  \
097-02                                              100.0    

        Case Conditions_C6P- in Heart Cut (Input)_wt%  \
097-02                                            1.0   

        Naphtha Properties_Oxygen_wt%  Naphtha Properties_Methane_wt%  \
097-02                            0.0                             0.0   

        Naphtha Properties_Ethane_wt%  Naphtha Properties_Propane_wt%  \
097-02                            0.0                           0.003   

        Naphtha Properties_n-Butane_wt%  Naphtha Properties_i-Pentane_wt%  \
097-02                            0.007                          0.057999   

        Naphtha Properties_n-Pentane_wt%  Naphtha Properties_tr2-Pentene_wt%  \
097-02                          0.119999                               0.001   

        Naphtha Properties_Cyclopentane_wt%  \
097-02                                 0.03   

        Naphtha Properties_3-Methylpentane_wt%  \
097-02                                0.816992   

        Naphtha Properties_tr2-Hexene_wt%  Naphtha Properties_n-Hexane_wt%  \
097-02                              0.017                         2.317977   

        Naphtha Properties_Methylcyclopentane_wt%  \
097-02                                   1.603984   

        Naphtha Properties_2,2-Dimethylpentane_wt%  \
097-02                                    0.624994   

        Naphtha Properties_Benzene_wt%  Naphtha Properties_Cyclohexane_wt%  \
097-02                        0.322997                            0.868991   

        Naphtha Properties_2,2,3-Trimethylbutane_wt%  \
097-02                                      0.686993   

        Naphtha Properties_4-Methyl-1-hexene_wt%  \
097-02                                     0.014   

        Naphtha Properties_2-Methylhexane_wt%  \
097-02                               5.202948   

        Naphtha Properties_t-1,3-Dimethylcyclopentane_wt%  \
097-02                                           0.806992   

        Naphtha Properties_n-Heptane_wt%  Naphtha Properties_H2O_wt%  \
097-02                          8.663913                         0.0   

        Naphtha Properties_Methylcyclohexane_wt%  \
097-02                                  4.671953   

        Naphtha Properties_Ethylcyclopentane_wt%  \
097-02                                  3.234968   

        Naphtha Properties_2,3-Dimethyl-1-hexene_wt%  \
097-02                                      0.093999   

        Naphtha Properties_Toluene_wt%  \
097-02                         3.02797   

        Naphtha Properties_2-Methylheptane_wt%  \
097-02                                9.296906   

        Naphtha Properties_n-Octane_wt%  \
097-02                         8.724913   

        Naphtha Properties_Ethylcyclohexane_wt%  \
097-02                                  7.09093   

        Naphtha Properties_EthylBenzene_wt%  Naphtha Properties_P-Xylene_wt%  \
097-02                             1.525985                         0.864991   

        Naphtha Properties_M-Xylene_wt%  Naphtha Properties_O-Xylene_wt%  \
097-02                         2.135978                         1.897981   

        Naphtha Properties_2-Methyloctane_wt%  \
097-02                               10.95889   

        Naphtha Properties_1-Nonene_wt%  Naphtha Properties_n-Nonane_wt%  \
097-02                         0.285997                         6.126939   

        Naphtha Properties_i-Propylbenzene_wt%  \
097-02                                0.055999   

        Naphtha Properties_1-trans-3,5-Trimethylcyclohexane_wt%  \
097-02                                           3.830962         

        Naphtha Properties_n-Propylcyclohexane_wt%  \
097-02                                    2.553974   

     

In [14]:
E_pred = E.predict(E_X)
E_pred

Duty_E062 3.5K Steam Rate_ton/hr  Duty_Reboiler Heat Duty_Mkcal/hr  \
0                           2.20551                          3.148844   

   NA_density  LE_density  HC_density  HE_density  
0     0.74304    0.692492    0.743165    0.764728

# F

In [15]:
F_X = D_pred[col_names['xhc']]
F_X

Heart Cut Product Properties_Oxygen_wt%  \
0                                      0.0   

   Heart Cut Product Properties_Methane_wt%  \
0                                       0.0   

   Heart Cut Product Properties_Ethane_wt%  \
0                                      0.0   

   Heart Cut Product Properties_Propane_wt%  \
0                                  0.000455   

   Heart Cut Product Properties_n-Butane_wt%  \
0                                   0.001922   

   Heart Cut Product Properties_i-Pentane_wt%  \
0                                    0.030148   

   Heart Cut Product Properties_n-Pentane_wt%  \
0                                    0.069311   

   Heart Cut Product Properties_tr2-Pentene_wt%  \
0                                       0.00091   

   Heart Cut Product Properties_Cyclopentane_wt%  \
0                                       0.016841   

   Heart Cut Product Properties_3-Methylpentane_wt%  \
0                                          0.374419   

   Heart Cut Product Properties_tr2-Hexene_wt%  \
0                                     0.014688   

   Heart Cut Product Properties_n-Hexane_wt%  \
0                                   0.988953   

   Heart Cut Product Properties_Methylcyclopentane_wt%  \
0                                           1.062583     

   Heart Cut Product Properties_2,2-Dimethylpentane_wt%  \
0                                            0.40567      

   Heart Cut Product Properties_Benzene_wt%  \
0                                  0.332756   

   Heart Cut Product Properties_Cyclohexane_wt%  \
0                                      0.874387   

   Heart Cut Product Properties_2,2,3-Trimethylbutane_wt%  \
0                                           0.517181        

   Heart Cut Product Properties_4-Methyl-1-hexene_wt%  \
0                                           0.013036    

   Heart Cut Product Properties_2-Methylhexane_wt%  \
0                                         5.558643   

   Heart Cut Product Properties_t-1,3-Dimethylcyclopentane_wt%  \
0                                           1.062583             

   Heart Cut Product Properties_n-Heptane_wt%  \
0                                    9.347732   

   Heart Cut Product Properties_H2O_wt%  \
0                                   0.0   

   Heart Cut Product Properties_Methylcyclohexane_wt%  \
0                                           6.387891    

   Heart Cut Product Properties_Ethylcyclopentane_wt%  \
0                                            4.42255    

   Heart Cut Product Properties_2,3-Dimethyl-1-hexene_wt%  \
0                                             0.1004        

   Heart Cut Product Properties_Toluene_wt%  \
0                                  4.019798   

   Heart Cut Product Properties_2-Methylheptane_wt%  \
0                                          9.937965   

   Heart Cut Product Properties_n-Octane_wt%  \
0                                   9.575292   

   Heart Cut Product Properties_Ethylcyclohexane_wt%  \
0                                           8.801903   

   Heart Cut Product Properties_EthylBenzene_wt%  \
0                                       1.718803   

   Heart Cut Product Properties_P-Xylene_wt%  \
0                                   0.967095   

   Heart Cut Product Properties_M-Xylene_wt%  \
0                                    2.38028   

   Heart Cut Product Properties_O-Xylene_wt%  \
0                                   1.990895   

   Heart Cut Product Properties_2-Methyloctane_wt%  \
0                                         9.600275   

   Heart Cut Product Properties_1-Nonene_wt%  \
0                                   0.228153   

   Heart Cut Product Properties_n-Nonane_wt%  \
0                                   4.630531   

   Heart Cut Product Properties_i-Propylbenzene_wt%  \
0                                          0.046953   

   Heart Cut Product Properties_1-trans-3,5-Trimethylcyclohexane_wt%  \
0                                            3.32224            

In [16]:
F_pred = F(F_X)
print(F_pred.sum(axis=1))
F_pred

0    100.0
dtype: float64


Hydrogen  Methane  Ethane   Propane  n-Butane  i-Butane  C4 Naphthenes  \
0       0.0      0.0     0.0  0.000455  0.000769  0.001057       0.000096   

   n-Pentane  C5 i-Paraffins  C5 Naphthenes  n-Hexane  C6 i-Paraffins  \
0   0.069311        0.031058       0.016841  0.988953        0.389107   

   C6 Naphthenes   Benzene  n-Heptane  C7 i-Paraffins  C7 Naphthenes  \
0        1.93697  0.332756   9.347732         6.49453      11.873025   

    Toluene  n-Octane  C8 i-Paraffins  C8 Naphthenes  Ethylbenzene  P-Xylene  \
0  4.019798  9.575292       10.038365       8.801903      1.718803  0.967095   

   M-Xylene  O-Xylene  n-Nonane  C9 i-Paraffins  C9 Naphthenes  C9 Aromtics  \
0   2.38028  1.990895  4.630531        9.828429       5.064498     3.023776   

   C10+ n-Paraffins  C10+ i-Paraffins  C10+ Naphthenes  C10+ Aromtics  
0          0.928856          4.371862         0.574356       0.602604

# G

In [17]:
G_pred = G(F_pred)
print(G_pred.sum(axis=1))
G_pred

0    100.0
dtype: float64


HYD  NC1  NC2       NC3       PC4       SAP        SAN       BEZ       TOL  \
0  0.0  0.0  0.0  0.000455  0.001826  56.69412  28.267593  0.332756  4.019798   

        ETB       PXY      MXY       OXY       C9A       C10  
0  1.718803  0.967095  2.38028  1.990895  3.023776  0.602604

# H

In [18]:
x = np.random.uniform(size=(1,len(H.x_col)))
x = pd.DataFrame(x,columns=H.x_col)
x

N+2A in Naphtha Feed, wt%      Chlorine in Fresh Catalyst, wt%  \
0                       0.444743                             0.036213   

   Carbon in Spent Catalyst, %      C6P- in Naphtha Feed, wt%  \
0                     0.718426                       0.149223   

       3R211 Inlet Tem, oC      3R212 Inlet Temp, oC  \
0                 0.613826                  0.359033   

       3R213 Inlet Temp, oC      3R214 Inlet Temp, oC  Feed    Hydrogen  \
0                  0.081897                  0.010182          0.958453   

   Feed    Methane  Feed    Ethane  Feed    Propane  Feed    n-Butane  \
0         0.501382        0.989481         0.541739          0.645078   

   Feed    i-Butane  Feed    C4 Naphthenes  Feed    n-Pentane  \
0           0.24378               0.944928           0.292023   

   Feed    C5 i-Paraffins  Feed    C5 Naphthenes  Feed    n-Hexane  \
0                 0.79608               0.202045          0.800622   

   Feed    C6 i-Paraffins  Feed    C6 Naphthenes  Feed    Benzene  \
0                0.776494               0.852915         0.108642   

   Feed    n-Heptane  Feed    C7 i-Paraffins  Feed    C7 Naphthenes  \
0           0.553933                0.714774               0.299208   

   Feed    Toluene  Feed    n-Octane  Feed    C8 i-Paraffins  \
0         0.207911          0.635169                0.739347   

   Feed    C8 Naphthenes  Feed    Ethylbenzene  Feed    P-Xylene  \
0               0.824548              0.118729          0.688581   

   Feed    M-Xylene  Feed    O-Xylene  Feed    n-Nonane  \
0           0.51459          0.812866          0.740326   

   Feed    C9 i-Paraffins  Feed    C9 Naphthenes  Feed    C9 Aromtics  \
0                0.071967               0.163289             0.047182   

   Feed    C10+ n-Paraffins  Feed    C10+ i-Paraffins  \
0                  0.715977                  0.180026   

   Feed    C10+ Naphthenes  Feed    C10+ Aromtics  \
0                 0.447285               0.611414   

   Product Separator Pressure, kg/cm2_g  H2/HC Ratio  
0                              0.179599     0.298542

In [19]:
H.predict(x)

Product    Hydrogen  Product    Methane  Product    Ethane  \
0             0.883202            0.177178           1.013763   

   Product    Propane  Product    n-Butane  Product    i-Butane  \
0            0.422334             0.136705             0.118882   

   Product    C4 Naphthenes  Product    n-Pentane  Product    C5 i-Paraffins  \
0                       0.0              0.274137                   0.431047   

   Product    C5 Naphthenes  Product    n-Hexane  Product    C6 i-Paraffins  \
0                  0.002139             0.137749                   0.339849   

   Product    C6 Naphthenes  Product    Benzene  Product    n-Heptane  \
0                  0.013517            0.375138               0.39937   

   Product    C7 i-Paraffins  Product    C7 Naphthenes  Product    Toluene  \
0                   1.401655                  0.020552            3.037438   

   Product    n-Octane  Product    C8 i-Paraffins  Product    C8 Naphthenes  \
0             0.137379                   0.079696                  0.018598   

   Product    Ethylbenzene  Product    P-Xylene  Product    M-Xylene  \
0                 0.774468             0.485922             3.271689   

   Product    O-Xylene  Product    n-Nonane  Product    C9 i-Paraffins  \
0             1.200721             0.000935                   0.004064   

   Product    C9 Naphthenes  Product    C9 Aromtics  \
0                  0.000343                2.288141   

   Product    C10+ n-Paraffins  Product    C10+ i-Paraffins  \
0                     0.002529                     0.016368   

   Product    C10+ Naphthenes  Product    C10+ Aromtics  
0                    0.004147                  0.271126

# I

In [20]:
A_pred = A.predict(A_X)
print(A_pred.sum(axis=1))
A_pred

0    100.0
dtype: float64


C5NP      C5IP       C5N      C6NP      C6IP       C6N       C6A  \
0  1.054952  0.713361  0.153138  3.288574  2.058669  3.550635  0.645131   

      C7NP      C7IP       C7N       C7A      C8NP      C8IP       C8N  \
0  7.78893  6.232049  9.894674  3.337994  7.951538  8.336734  7.354495   

        C8A     C9NP      C9IP       C9N       C9A     C10NP     C10IP  \
0  6.043764  5.17344  9.482511  5.116143  3.679991  1.219736  5.392507   

       C10N      C10A  
0  0.730878  0.800157

In [21]:
B_pred = B(A_pred)
print(B_pred.sum(axis=1))
B_pred.head()

0    100.0
dtype: float64


O2    METHANE    ETHANE    PROPANE    BUTANE    IPENTANE    PENTANE  \
0   0.0        0.0       0.0   0.035668  0.071336     0.59209   1.054952   

     T2PENTEN        CP       3MP    T2HEXENE    HEXANE       MCP      22MP  \
0    0.014267  0.153138  2.017495    0.041173  3.288574  2.303498  0.596948   

     BENZENE        CH     223MB    4M1HEXEN      2MHX      1T3M    HEPTANE  \
0   0.645131  1.247137  0.655922    0.013084  4.966095  0.915905    7.78893   

     H2O       MCH      ECP    23DM1HX    TOLUENE    2MHEPT    OCTANE  \
0    0.0  5.305518  3.67325   0.083367   3.337994  8.253367  7.951538   

        ECH    EBENZENE    PXYLENE    MXYLENE    OXYLENE    2MOCTN    1NONENE  \
0  7.354495    1.435122   0.814092   2.009385   1.785166   9.24163   0.240881   

     NONANE    CUMENE    1T35TMCH       PCH    PRBENZEN    135MBENZ  \
0   5.17344  0.052158    3.069545  2.046597    0.222474    0.955408   

     1M2EBNZN    123MBENZ    24DMOCT      TEMH       3MN    TBCYHXAN  \
0    1.411984    1.037967   0.906823  1.345499  3.140185    0.730878   

     DECANE     ODEBZ    UNDECANE    PNBZ    DODECANE    HXBNZN    TRIDECAN  
0  1.022837  0.748226    0.193451  0.0435    0.003449   0.00843         0.0

In [22]:
I(B_pred)

C5NP      C5IP       C5N      C6NP      C6IP       C6N       C6A  \
0  1.054952  0.713361  0.153138  3.288574  2.058669  3.550635  0.645131   

      C7NP      C7IP       C7N       C7A      C8NP      C8IP       C8N  \
0  7.78893  6.232049  9.894674  3.337994  7.951538  8.336734  7.354495   

        C8A     C9NP      C9IP       C9N       C9A     C10NP     C10IP  \
0  6.043764  5.17344  9.482511  5.116143  3.679991  1.219736  5.392507   

       C10N      C10A  
0  0.730878  0.800157

# 把全部模組整合到一塊

In [23]:
class EVA(object):
    def __init__(self):
        self.A = joblib.load('./model/SVR(4_to_23).pkl')
        self.B = joblib.load('./model/transformer(23_to_54).pkl')
        self.C = joblib.load('./model/ANN(58_to_sp162).pkl')
        self.D = joblib.load('./model/transformer(SP162_to_Y162).pkl')
        self.E = joblib.load('./model/ANN(energy).pkl')
        self.F = joblib.load('./model/transformer(54_to_33).pkl')
        self.G = joblib.load('./model/transformer(33_to_15).pkl')
        self.H = joblib.load('./model/transformer(43_to_33).pkl')
        self.I = joblib.load('./model/transformer(54_to_23).pkl')
        
        self.J = joblib.load('./model/ANN(58_to_y162).pkl') #2020/07/02新增
        
        self.col_names = joblib.load('./data/phase_2/cleaned/col_names.pkl')

    def __call__(self, X):
        x4, case4 = X.iloc[:, :4], X.iloc[:, 4:]
        # model forward
        self.y23 = self.A.predict(x4)  # percent sum = 100
        self.Xna = self.B(self.y23)  # percent sum = 100
        # percent sum = 1 * 54component = 54
        self.sp162 = self.C.predict(case4.join(self.Xna))
        # percent sum = 100*3(le,hc,he) = 300
        
        self.y162 = self.J.predict(case4.join(self.Xna)) # 2020/07/02新增
        
        self.xhc = self.y162[self.col_names['xhc']]  # percent sum = 100
        self.xhc33_p = self.F(self.xhc)/100  # percent

        # predict duty and density(ton/m3)
        E_pred = self.E.predict(case4.join(self.Xna))
        self.duty = E_pred.iloc[:, :2]
        self.density = E_pred.iloc[:, 2:]

        # get split factor
        self.shc = self.sp162[self.col_names['shc']]
        self.sle = self.sp162[self.col_names['sle']]
        self.she = self.sp162[self.col_names['she']]

        # fhc_ton = fhc_m3*density
        self.fhc_m3 = case4['Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr']
        self.fhc_ton = self.fhc_m3.values * self.density.iloc[:, 2].values

        # calculate fle_ton,fhe_ton
        self.fle_ton = self.fhc_ton * ((self.Xna.values@self.sle.values.ravel()) /
                                       (self.Xna.values@self.shc.values.ravel()))
        self.fhe_ton = self.fhc_ton * ((self.Xna.values@self.she.values.ravel()) /
                                       (self.Xna.values@self.shc.values.ravel()))

        # calculate fna
        self.fna_ton = self.fle_ton + self.fhc_ton + self.fhe_ton

        # scale fle,fhc,fhe
        self.fle_ton = self.fle_ton*100000/self.fna_ton
        self.fhc_ton = self.fhc_ton*100000/self.fna_ton
        self.fhe_ton = self.fhe_ton*100000/self.fna_ton

        # add columns name
        self.fle_ton = pd.Series(
            self.fle_ton, name=self.col_names['Rate_ton'][1])
        self.fhe_ton = pd.Series(
            self.fhe_ton, name=self.col_names['Rate_ton'][3])

        # scale duty
        self.duty *= 100/self.fna_ton[0]

        # xhc33 = fhc_ton*xhc_33_p
        self.xhc33 = self.fhc_ton[0]*self.xhc33_p
        self.xhc33_tmp = self.xhc33*74000/self.fhc_ton[0]

        # xhe33 = H(someting + xhc33 + something)
        xhc23 = self.I(self.xhc)
        C6Pm = xhc23[['C5NP', 'C5IP', 'C6IP', 'C6NP']].sum(axis=1)[0]

        NpA = xhc23[['C5N', 'C6N', 'C7N', 'C8N', 'C9N', 'C10N']].sum(axis=1)[0] +\
            xhc23[['C6A', 'C7A', 'C8A', 'C9A', 'C10A']].sum(axis=1)[0]*2

        self.H_input = [NpA, 0.942, 3.78, C6Pm, 517, 517, 517, 517] + \
            self.xhc33_tmp.values.ravel().tolist()+[4.798, 1.44]
        self.H_input = np.array([self.H_input])
        self.H_input = pd.DataFrame(self.H_input, columns=self.H.x_col)
        self.reformer_input = [NpA, 0.942, 3.78, C6Pm, 517, 517, 517, 517] + self.xhc33.values.ravel().tolist() + [4.798, 1.44]
        self.reformer_input = np.array([self.reformer_input])
        self.reformer_input = pd.DataFrame(self.reformer_input, columns=self.H.x_col)

        self.重組33_tmp = self.H.predict(self.H_input)

        self.重組33 = self.重組33_tmp *self.fhc_ton[0]/74000

        # final output
        self.重組15 = self.G(self.重組33)

        # raname and combine return to user
        self.predict = self.重組15.join(self.fle_ton).join(
            self.fhe_ton).join(self.duty)
        self.naphtha = self.y23
        self.pre_d = self.sp162
        self.reform = self.重組33

In [24]:
eva = EVA()

# save eva

In [25]:
joblib.dump(eva,'./model/eva.pkl')
del eva

# load eva

In [26]:
eva = joblib.load('./model/eva.pkl')

# create input X for eva

In [27]:
X = pd.DataFrame(np.hstack((A_X.values,case.values)))
X.columns = A_X.columns.tolist()+case.columns.tolist()
X.head()

T10    T50    T90     N+A  \
0  99.9  119.9  146.9  41.307   

   Case Conditions_Feed Rate to C013 (Input)_m3/hr  \
0                                             20.0   

   Case Conditions_C7+ in Light End (Input)_wt%  \
0                                          10.0   

   Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr  \
0                                              100.0    

   Case Conditions_C6P- in Heart Cut (Input)_wt%  
0                                            1.5

# get eva result

In [28]:
eva(X)

[[9.99764860e-01 3.16569989e-04 2.89963122e-04 9.99631882e-01
  1.98322203e-04 2.36381791e-04 9.99775112e-01 2.26373508e-04
  2.54330196e-04 9.89390314e-01 1.05778323e-02 1.10683664e-04
  9.77368951e-01 2.23548524e-02 7.06154970e-05 9.56218898e-01
  4.21928540e-02 5.95514684e-05 9.44880903e-01 5.44991009e-02
  1.35346709e-04 9.49462891e-01 5.30663654e-02 1.03676000e-04
  9.08061922e-01 9.12098289e-02 1.01135724e-04 8.44387889e-01
  1.53744787e-01 7.92755091e-05 7.06775248e-01 2.97280520e-01
  3.93984155e-05 7.48252213e-01 2.48875275e-01 6.96869174e-05
  6.14068925e-01 3.80927294e-01 2.97770330e-05 4.34244663e-01
  5.61812580e-01 8.65378461e-05 5.72579980e-01 4.28726554e-01
  2.35613377e-04 4.17436838e-01 5.81060350e-01 1.20004239e-04
  3.43997002e-01 6.51580513e-01 1.60790936e-04 1.74383685e-01
  8.33249450e-01 1.27568055e-04 7.10266978e-02 9.29326355e-01
  6.59439611e-05 3.69998030e-02 9.63623345e-01 2.11677572e-04
  3.89123918e-03 9.98588264e-01 1.15099429e-04 9.99701023e-01
  2.0852

In [29]:
eva.predict

HYD         NC1          NC2          NC3          PC4  \
0  2876.055784  867.628757  2941.984216  3440.852973  1460.874486   

            SAP         SAN          BEZ           TOL          ETB  \
0  15309.465056  742.025429  2396.835027  15757.580108  3708.220541   

           PXY           MXY          OXY           C9A          C10  \
0  2778.123243  10957.462486  5541.335784  11574.927568  2637.933189   

   Light End Product Properties_Light End Prod. Rate_ton/hr  \
0                                        9260.837754          

   Heavy End Product Properties_Heavy End Product Rate_ton/hr  \
0                                        7747.855781            

   Duty_E062 3.5K Steam Rate_ton/hr  Duty_Reboiler Heat Duty_Mkcal/hr  
0                          2.592368                          4.132529

In [30]:
eva.xhc33

Hydrogen  Methane  Ethane   Propane  n-Butane  i-Butane  C4 Naphthenes  \
0       0.0      0.0     0.0  0.409456  0.723842  0.995283        0.09048   

   n-Pentane  C5 i-Paraffins  C5 Naphthenes    n-Hexane  C6 i-Paraffins  \
0   62.54957       29.352557      13.551176  827.243967      323.504373   

   C6 Naphthenes     Benzene    n-Heptane  C7 i-Paraffins  C7 Naphthenes  \
0    1584.549444  266.602874  7754.693767     5357.187622    9695.791033   

       Toluene     n-Octane  C8 i-Paraffins  C8 Naphthenes  Ethylbenzene  \
0  3280.152012  8022.680688     8423.050815    7167.135113   1469.150613   

     P-Xylene     M-Xylene     O-Xylene     n-Nonane  C9 i-Paraffins  \
0  826.922038  2039.288522  1670.440309  3844.627352     8230.931503   

   C9 Naphthenes  C9 Aromtics  C10+ n-Paraffins  C10+ i-Paraffins  \
0    4157.829552  2543.913961        742.243355       3698.481577   

   C10+ Naphthenes  C10+ Aromtics  
0       472.404474     484.801996

In [31]:
eva.naphtha

C5NP      C5IP       C5N      C6NP      C6IP       C6N       C6A  \
0  1.054952  0.713361  0.153138  3.288574  2.058669  3.550635  0.645131   

      C7NP      C7IP       C7N       C7A      C8NP      C8IP       C8N  \
0  7.78893  6.232049  9.894674  3.337994  7.951538  8.336734  7.354495   

        C8A     C9NP      C9IP       C9N       C9A     C10NP     C10IP  \
0  6.043764  5.17344  9.482511  5.116143  3.679991  1.219736  5.392507   

       C10N      C10A  
0  0.730878  0.800157

In [32]:
eva.pre_d

Individual Component to Light End Split Factor_Oxygen_Ratio  \
0                                           0.999394             

   Individual Component to Heart Cut Split Factor_Oxygen_Ratio  \
0                                           0.000316             

   Individual Component to Heavy End Split Factor_Oxygen_Ratio  \
0                                            0.00029             

   Individual Component to Light End Split Factor_Methane_Ratio  \
0                                           0.999565              

   Individual Component to Heart Cut Split Factor_Methane_Ratio  \
0                                           0.000198              

   Individual Component to Heavy End Split Factor_Methane_Ratio  \
0                                           0.000236              

   Individual Component to Light End Split Factor_Ethane_Ratio  \
0                                            0.99952             

   Individual Component to Heart Cut Split Factor_Ethane_Ratio  \
0                                           0.000226             

   Individual Component to Heavy End Split Factor_Ethane_Ratio  \
0                                           0.000254             

   Individual Component to Light End Split Factor_Propane_Ratio  \
0                                           0.989312              

   Individual Component to Heart Cut Split Factor_Propane_Ratio  \
0                                           0.010577              

   Individual Component to Heavy End Split Factor_Propane_Ratio  \
0                                           0.000111              

   Individual Component to Light End Split Factor_n-Butane_Ratio  \
0                                            0.97757               

   Individual Component to Heart Cut Split Factor_n-Butane_Ratio  \
0                                           0.022359               

   Individual Component to Heavy End Split Factor_n-Butane_Ratio  \
0                                           0.000071               

   Individual Component to Light End Split Factor_i-Pentane_Ratio  \
0                                           0.957683                

   Individual Component to Heart Cut Split Factor_i-Pentane_Ratio  \
0                                           0.042257                

   Individual Component to Heavy End Split Factor_i-Pentane_Ratio  \
0                                            0.00006                

   Individual Component to Light End Split Factor_n-Pentane_Ratio  \
0                                           0.945339                

   Individual Component to Heart Cut Split Factor_n-Pentane_Ratio  \
0                                           0.054526                

   Individual Component to Heavy End Split Factor_n-Pentane_Ratio  \
0                                           0.000135                

   Individual Component to Light End Split Factor_tr2-Pentene_Ratio  \
0                                            0.94697                  

   Individual Component to Heart Cut Split Factor_tr2-Pentene_Ratio  \
0                                           0.052927                  

   Individual Component to Heavy End Split Factor_tr2-Pentene_Ratio  \
0                                           0.000103                  

   Individual Component to Light End Split Factor_Cyclopentane_Ratio  \
0                                           0.908632                   

   Individual Component to Heart Cut Split Factor_Cyclopentane_Ratio  \
0                                           0.091267                   

   Individual Component to Heavy End Split Factor_Cyclopentane_Ratio  \
0                                           0.000101                   

   Individual Component to Light End Split Factor_3-Methylpentane_Ratio  \
0                                             0.8459                      

   Individual Component to Heart Cut Split Factor_3-Methylpentane_Ratio  \
0                                            0.15402        

In [33]:
eva.reform

Product    Hydrogen  Product    Methane  Product    Ethane  \
0          2876.055784          867.628757        2941.984216   

   Product    Propane  Product    n-Butane  Product    i-Butane  \
0         3440.852973           809.454811           651.419676   

   Product    C4 Naphthenes  Product    n-Pentane  Product    C5 i-Paraffins  \
0                  1.029733           2186.179459                   3365.064   

   Product    C5 Naphthenes  Product    n-Hexane  Product    C6 i-Paraffins  \
0                 35.404267           648.545676                1552.685514   

   Product    C6 Naphthenes  Product    Benzene  Product    n-Heptane  \
0                108.613696         2396.835027           1222.027568   

   Product    C7 i-Paraffins  Product    C7 Naphthenes  Product    Toluene  \
0                4683.534703                208.183243        15757.580108   

   Product    n-Octane  Product    C8 i-Paraffins  Product    C8 Naphthenes  \
0           291.272865                 913.104324                202.172622   

   Product    Ethylbenzene  Product    P-Xylene  Product    M-Xylene  \
0              3708.220541          2778.123243         10957.462486   

   Product    O-Xylene  Product    n-Nonane  Product    C9 i-Paraffins  \
0          5541.335784            21.299404                  85.533959   

   Product    C9 Naphthenes  Product    C9 Aromtics  \
0                 29.044088            11574.927568   

   Product    C10+ n-Paraffins  Product    C10+ i-Paraffins  \
0                   111.764122                    227.42373   

   Product    C10+ Naphthenes  Product    C10+ Aromtics  
0                  158.607514               2637.933189